In [1]:
%run SAT_help_functions.ipynb

## OUM

In [2]:
data_dir = "/Users/wangjiashun/Desktop/Reserach on SAT/datasets/Evans2021/Rae2014/dataBySubject/sub"
sample_size = 34
SAT_conds = ["Speed", "Accuracy"]

In [3]:
class DriftBothSpeedAcc(Drift):
    name = "DriftBothSpeedAcc"
    required_parameters = ["Vspeed","Vacc", "Kspeed", "Kacc"]
    required_conditions = ['Cond']
        
    def get_drift(self, x, t, conditions, **kwargs):
        if conditions['Cond'] == 'Speed':
            return self.Vspeed + x*self.Kspeed 
        elif conditions['Cond'] == 'Accuracy':
            return self.Vacc + x*self.Kacc
        else: 
            print("Non-implemented SAT conditions")

In [4]:
class NondecisionSpeedAcc(OverlayNonDecision):
    name = "DriftSpeedAcc"
    required_parameters = ["tSpeed", 'tAcc']
    required_conditions = ['Cond']
        
    def get_nondecision_time(self, conditions, **kwargs):
        if conditions['Cond'] == 'Speed':
            return self.tSpeed 
        elif conditions['Cond'] == 'Accuracy':
            return self.tAcc
        else: 
            print("Non-implemented SAT conditions")

In [9]:
def get_OUM_result_sub_2v2k2ter(samp, v = [0,6], a = [0.5,8], k = [0,15], ter=[0,1], 
                                dx=.01, dt=.01, T_dur=5):
    
    model_fit = Model(name='Simple model (fitted)',
                      drift=DriftBothSpeedAcc(Vspeed = Fittable(minval=v[0], maxval=v[1]),
                                              Vacc = Fittable(minval=v[0], maxval=v[1]),
                                              Kspeed=Fittable(minval=k[0], maxval=k[1]),
                                              Kacc=Fittable(minval=k[0], maxval=k[1])), 
                      noise=NoiseConstant(noise=1),
                      bound=BoundConstant(B=Fittable(minval=a[0], maxval=a[1])),                      
                      overlay=NondecisionSpeedAcc(
                          tSpeed=Fittable(minval=ter[0], maxval=ter[1]),
                          tAcc=Fittable(minval=ter[0], maxval=ter[1]),

                      ),
                      dx=dx, dt=dt, T_dur=T_dur)
    #set_N_cpus(4)
    fit_adjust_model(samp, model_fit,
                     fitting_method="differential_evolution",
                     lossfunction=LossRobustBIC, verbose=False)
    #print(model_fit.get_model_parameters())
    df = pd.DataFrame(data = np.array(model_fit.get_model_parameters()).reshape(1,7),
                      columns = model_fit.get_model_parameter_names())
    
    #df['Participant']=i+1
    #df['SAT'] = SAT
    df['BIC'] = model_fit.get_fit_result().value()
    df = df.rename(columns={'Vspeed': 'Drift Rate Speed',
                            'Vacc': 'Drift Rate Accuracy',
                            'B': 'Boundary Separation',
                            'Kspeed': "Self-excitation Speed",
                            'Kacc': "Self-excitation Accuracy",
                            'tSpeed': 'Non-decision Time Speed',
                            'tAcc': 'Non-decision Time Accuracy'})
    return df

In [ ]:
df_all = pd.DataFrame()
for i in range(sample_size):
    print(i)
    path = os.path.join(data_dir+str(i+1)+".csv")
    df = pd.read_csv(path, sep = ",")
    #df = df[df["cond"]==SAT] # A condition
    df = df[df["Time"]<5]
    #df = df[df["Time"]>0.2]

    samp = Sample.from_pandas_dataframe(df, rt_column_name="Time", correct_column_name="Correct")
    df = get_OUM_result_sub_2v2k2ter(samp,dx=.001, dt=.0001)
    df_all = pd.concat((df_all,df))
    print(df)
    
df_OUM = df_all.copy()

0


Info: Params [1.15587714 1.78262926 6.50463009 5.43243675 1.80912826 0.13417938
 0.2139502 ] gave -987.3780727786509


   Drift Rate Speed  Drift Rate Accuracy  Self-excitation Speed  \
0          1.155877             1.782629                6.50463   

   Self-excitation Accuracy  Boundary Separation  Non-decision Time Speed  \
0                  5.432437             1.809128                 0.134179   

   Non-decision Time Accuracy         BIC  
0                     0.21395 -987.378073  
1


Info: Params [ 0.73826583  1.47141979 10.14665196  9.0714633   3.15999139  0.12424339
  0.17625669] gave -2141.040797657148


   Drift Rate Speed  Drift Rate Accuracy  Self-excitation Speed  \
0          0.738266              1.47142              10.146652   

   Self-excitation Accuracy  Boundary Separation  Non-decision Time Speed  \
0                  9.071463             3.159991                 0.124243   

   Non-decision Time Accuracy          BIC  
0                    0.176257 -2141.040798  
2


...

   Drift Rate Speed  Drift Rate Accuracy  Self-excitation Speed  \
0          1.022759             1.126578               7.915725   

   Self-excitation Accuracy  Boundary Separation  Non-decision Time Speed  \
0                  8.348448             2.971814                 0.167451   

   Non-decision Time Accuracy         BIC  
0                    0.196677 -1346.57835  
15


Info: Params [0.02253296 0.53793524 7.86738982 5.56809022 1.03232051 0.14429512
 0.15598009] gave 191.9678841631109


   Drift Rate Speed  Drift Rate Accuracy  Self-excitation Speed  \
0          0.022533             0.537935                7.86739   

   Self-excitation Accuracy  Boundary Separation  Non-decision Time Speed  \
0                   5.56809             1.032321                 0.144295   

   Non-decision Time Accuracy         BIC  
0                     0.15598  191.967884  
16


Info: Params [ 0.31545872  0.42999193 10.74962828  8.99871171  3.21991231  0.03656168
  0.04054824] gave -1446.012405288175


   Drift Rate Speed  Drift Rate Accuracy  Self-excitation Speed  \
0          0.315459             0.429992              10.749628   

   Self-excitation Accuracy  Boundary Separation  Non-decision Time Speed  \
0                  8.998712             3.219912                 0.036562   

   Non-decision Time Accuracy          BIC  
0                    0.040548 -1446.012405  
17


In [ ]:
df_OUM.to_csv("OUM2v2k2terresultRae2014_large_k_small_dt.csv")

In [7]:
df_OUM["BIC"].mean()

-827.3177340922819